In [1]:
import os
os.chdir('..')
os.getcwd()

'd:\\Programming\\PROJECTS\\Data Science\\FSDS\\CNN_Classifier_MLOPS'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from deepclassifier.constants import *
from deepclassifier.utils import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAM_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        params = self.params
        create_directories(
            [Path(config.tensorboard_root_log_dir), Path(os.path.dirname(config.checkpoint_model_filepath))]
        )
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
        )
        return prepare_callbacks_config


In [9]:
import time
import tensorflow as tf

class PrepareCallbacks:
    def __init__(self,config:PrepareCallbacksConfig):
        self.config=config
    
    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log=os.path.join(self.config.tensorboard_root_log_dir,f'tb_logs_at_{timestamp}')
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [10]:
try:
    config=ConfigurationManager()
    prepare_callbacks_config=config.get_prepare_callbacks_config()
    prepare_callbacks=PrepareCallbacks(prepare_callbacks_config)
    prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2022-09-21 00:47:58,763] deepclassifierLogger:INFO:: common:common.py:read_yaml() => yaml file: configs\config.yaml loaded successfully
[2022-09-21 00:47:58,766] deepclassifierLogger:INFO:: common:common.py:read_yaml() => yaml file: params.yaml loaded successfully
[2022-09-21 00:47:58,777] deepclassifierLogger:INFO:: common:common.py:create_directories() => created directory at: artifacts
[2022-09-21 00:47:58,779] deepclassifierLogger:INFO:: common:common.py:create_directories() => created directory at: artifacts\prepare_callbacks\tensorboard_log_dir
[2022-09-21 00:47:58,781] deepclassifierLogger:INFO:: common:common.py:create_directories() => created directory at: artifacts\prepare_callbacks\checkpoint_dir
